In [1]:
import numpy as np
import pandas as pd
from lime.lime_tabular import LimeTabularExplainer
import joblib
import os
from sklearn.model_selection import train_test_split

In [2]:
path=os.path.join("..", "app", "model.joblib")
best_pipeline = joblib.load(path)

path=os.path.join("..","data", "processed", "telco_churn_cleaned.csv")
df=pd.read_csv(path, delimiter=",")

y=df['Churn']
X=df.drop(columns=['Churn'])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=42
)

In [ ]:
cat_cols = X_train.select_dtypes(exclude=[np.number]).columns.tolist()
num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

cat_idx = [X_train.columns.get_loc(c) for c in cat_cols]
cat_names = {
    X_train.columns.get_loc(c): sorted(X_train[c].dropna().unique().tolist())
    for c in cat_cols
}
'''
explainer = LimeTabularExplainer(
    training_data=X_train.values,
    feature_names=X_train.columns.tolist(),
    class_names=['No churn','Churn'],
    categorical_features=cat_idx,
    categorical_names=cat_names,
    discretize_continuous=True,
    random_state=42,
    mode='classification'
)

def predict_fn(x):
    df_ = pd.DataFrame(x, columns=X_train.columns)
    return best_pipeline.predict_proba(df_)
'''

ValueError: could not convert string to float: 'Male'